<a href="https://colab.research.google.com/github/Gayaz2000/Redis_Test/blob/main/redis_caching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain_community langchain_core langchain_redis redis langchain_groq langchain_ollama

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.7/278.7 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.3/152.3 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.7 MB/s eta 0:00:00


In [2]:
import os
import getpass

os.environ["GROQ_API_KEY"] = getpass.getpass("Enter Groq api key: ")

Enter Groq api key: ··········


### Load dependecies

In [3]:
import time
import redis

from langchain_core.globals import set_llm_cache
from langchain_groq import ChatGroq
from langchain_redis import RedisCache, RedisSemanticCache
from langchain_ollama import OllamaEmbeddings

### Connecting to Redis

In [ ]:
REDIS_URL = getpass.getpass("Enter Redis URL: ") #"redis://localhost:6379"
redis_client = redis.from_url(REDIS_URL)
redis_client.ping()

### Using Redis for Standard Cache

In [ ]:
redis_cache = RedisCache(redis_url=REDIS_URL)
set_llm_cache(redis_cache)

llm = ChatGroq(model="gemma2-9b-it")

def execute_with_timing(prompt):
  start_time = time.time()
  result = llm.invoke(prompt)
  end_time = time.time()
  return result, end_time - start_time

#### Usecase

In [ ]:
prompt = ""
result, time_taken = execute_with_timing(prompt)
print(f"Result: {result}")
print(f"Time taken: {time_taken}")

In [ ]:
prompt = ""
result, time_taken = execute_with_timing(prompt)
print(f"Result: {result}")
print(f"Time taken: {time_taken}")

In [ ]:
# redis_cache.clear() # removes stored cache

### Using Redis Semantic Caching

In [ ]:
embedding = OllamaEmbeddings(model="llama3.2:1b")
redis_semantic_cache = RedisSemanticCache(
    embeddings= embedding,
    redis_url=REDIS_URL,
    distance_threshold= 0.2
    )

set_llm_cache(redis_semantic_cache)

#### Usecase

In [ ]:
prompt = ""
result, time_taken = execute_with_timing(prompt)
print(f"Result: {result}")
print(f"Time taken: {time_taken}")

In [ ]:
prompt = ""
result, time_taken = execute_with_timing(prompt)
print(f"Result: {result}")
print(f"Time taken: {time_taken}")

In [ ]:
# redis_semantic_cache.clear()

## Chat Message History using redis

In [ ]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_redis import RedisChatMessageHistory

session_id = ""

redis_message_history = RedisChatMessageHistory(
    session_id=session_id,
    redis_url=REDIS_URL,
)

In [ ]:
redis_message_history.add_user_message("Hi, AI system")
redis_message_history.add_ai_message("Hello!, How can I help you.") # Response for query

In [ ]:
redis_message_history.messages

### Chatbot integration

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

chain = prompt | llm

In [ ]:
def get_redis_chat_history(session_id: str)-> BaseChatMessageHistory:
  return RedisChatMessageHistory(session_id=session_id, redis_url= REDIS_URL)

In [ ]:
chain_with_chat_history = RunnableWithMessageHistory(
    chain,
    get_session_history= get_redis_chat_history,
    input_messages_key= "input",
    history_messages_key="history"
)

response = chain_with_chat_history.invoke(
    {"input": "Hi, AI system"},
    config={"configurable":{"session_id":"1"}}
)
print(response)

In [ ]:
redis_message_history.clear()

## Redis Vector DB

In [ ]:
import redis

#REDIS_URL = getpass.getpass("Enter Redis URL: ") #"redis://localhost:6379"
redis_client = redis.Redis(host="localhost", port= 6379, decode_responses= True)
redis_client.ping()

Test

In [ ]:
bike067 = redis_client.json().get("redisbikesco:bike:rbc00067")
bike067

In [ ]:
sample_description = bike067["description"]
sample_description

In [ ]:
text_embeddings = embedding.embed_query([sample_description])
text_embeddings

In [ ]:
len(text_embeddings)

Embedding Creation

In [ ]:
keys = redis_client.keys("redisbikesco:bike:*")
keys[:3]

In [ ]:
len(keys)

In [ ]:
import numpy as np

descriptions = redis_client.json().mget(keys, "$.description")
descriptions = [item for sublist in descriptions for item in sublist]
descriptions[:3]

In [ ]:
embeddings = embedding.embed_documents(descriptions)
len(embeddings)

In [ ]:
pipeline = redis_client.pipeline()

for key, embedding in zip(keys, embeddings):
  pipeline.json().set(key, "$.desc_embedding", embedding)

pipeline.execute()

In [ ]:
import json

json.dumps(redis_client.json().get("redisbikesco:bike:rbc00001"), indent=2)

### Index Building in Redis

In [ ]:
from redis.commands.search.index_definition import IndexDefinition, IndexType
from redis.commands.search.field import TagField, TextField, NumericField, VectorField
from redis.commands.search.query import Query

INDEX_NAME = "idx:bikes_vss"
DOC_PREFIX = "redisbikesco:bike:"
VECTOR_DIMENSION = 1536

In [ ]:
## The search index schema

try:
    redis_client.ft(INDEX_NAME).info()
    print('Index already exists!')
except:
    schema = (
        TextField('$.model', no_stem=True, as_name='model'),
        TextField('$.brand', no_stem=True, as_name='brand'),
        NumericField('$.price', as_name='price'),
        TagField('$.type', as_name='type'),
        TextField('$.description', as_name='description'),
        VectorField('$.description_embeddings', 'FLAT', {
            'TYPE': 'FLOAT32',
            'DIM': VECTOR_DIMENSION,
            'DISTANCE_METRIC': 'COSINE',
        }, as_name='vector'),
    )

    # Index Definition
    definition = IndexDefinition(prefix=[DOC_PREFIX], index_type=IndexType.JSON)

    # Create Index
    redis_client.ft(INDEX_NAME).create_index(fields=schema, definition=definition)

In [ ]:
query = (
    Query("@bikes:Montain Bikes").return_fields("id","price","brand","description")
)

redis_client.ft(INDEX_NAME).search(query).docs

In [ ]:
## Semantic query

In [ ]:
## KNN Query